In [1]:
from core.db import get_engine
from sqlalchemy import text

contract_codes = ['GT0037', 'US0094', 'US0116', 'US0117']
inventory_year = 2025

engine = get_engine()
default_values = {
    "inventory_year": inventory_year,
    "inventory_date": "pending",
    "rel_path": "",
}

for code in contract_codes:
    pkid = f"{code} {inventory_year}"  # o usa '_' si prefieres: f"{code}_{inventory_year}"
    q = text("""
        INSERT INTO masterdatabase.inventory_metrics
            (contract_code, inventory_year, inventory_date, rel_path, pkid)
        VALUES
            (:contract_code, :inventory_year, :inventory_date, :rel_path, :pkid)
        ON CONFLICT (pkid) DO NOTHING
    """)
    with engine.begin() as conn:
        conn.execute(q, {
            "contract_code": code,
            "pkid": pkid,
            **default_values
        })
    print(f"✅ Insertado {code} para el año {inventory_year}")


💻 Conectado a la base de datos helloworldtree
✅ Insertado GT0037 para el año 2025
✅ Insertado US0094 para el año 2025
✅ Insertado US0116 para el año 2025
✅ Insertado US0117 para el año 2025


In [ ]:
from core.db import get_engine
from sqlalchemy import text

contract_codes = ['GT0037', 'US0094', 'US0116', 'US0117']
inventory_year = 2024

engine = get_engine()
default_values = {
    "inventory_year": inventory_year,
    "inventory_date": "pending",
    "rel_path": "",
}

for code in contract_codes:
    pkid = f"{code} {inventory_year}"  # o usa '_' si prefieres: f"{code}_{inventory_year}"
    q = text("""
        INSERT INTO masterdatabase.inventory_metrics
            (contract_code, inventory_year, inventory_date, rel_path, pkid)
        VALUES
            (:contract_code, :inventory_year, :inventory_date, :rel_path, :pkid)
        ON CONFLICT (pkid) DO NOTHING
    """)
    with engine.begin() as conn:
        conn.execute(q, {
            "contract_code": code,
            "pkid": pkid,
            **default_values
        })
    print(f"✅ Insertado {code} para el año {inventory_year}")


In [3]:
import json
from core.db import get_engine
from sqlalchemy import text
import pandas as pd

# --- Configura el batch y año que quieres checar ---
BATCH_PATH = r"C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\Cruises\batch_imports.json"
AÑO = 2024

# 1. Carga todos los contract_codes + rel_paths de batch_imports.json
with open(BATCH_PATH, encoding="utf-8") as f:
    lotes = json.load(f)

# Ajusta a tu estructura: aquí asumo que cada lote tiene archivos de un solo país/año
# Si tienes varios, filtra solo el lote que quieres
todos = []
for lote in lotes:
    if str(lote.get("año")) == str(AÑO):
        archivos = lote["archivos"]
        for a in archivos:
            # Asume formato: .../GT0037_algo.xlsx
            name = a.split('/')[-1].split('\\')[-1]
            contract_code = name.split('_')[0]
            todos.append({"contract_code": contract_code, "rel_path": a})
# Si hay duplicados, deja solo uno por contract_code:
batch_df = pd.DataFrame(todos).drop_duplicates("contract_code")

# 2. Consulta los contract_codes ya existentes en inventory_metrics para ese año
engine = get_engine()
sql = """
SELECT contract_code
FROM masterdatabase.inventory_metrics
WHERE inventory_year = :año
"""
existentes = pd.read_sql(text(sql), engine, params={"año": AÑO})

# 3. Diferencial: los que están en batch pero no en la tabla
faltantes = batch_df[~batch_df['contract_code'].isin(existentes['contract_code'])]

print(f"Total en batch: {len(batch_df)}")
print(f"Ya existen en DB: {len(existentes)}")
print(f"Faltan por insertar: {len(faltantes)}")
print(faltantes)

# 4. Inserta los faltantes con rel_path correcto
for _, row in faltantes.iterrows():
    code = row["contract_code"]
    rel_path = row["rel_path"]
    pkid = f"{code} {AÑO}"  # Si usas pkid, ajústalo aquí

    q = text("""
        INSERT INTO masterdatabase.inventory_metrics
            (contract_code, inventory_year, inventory_date, rel_path, pkid)
        VALUES
            (:contract_code, :inventory_year, :inventory_date, :rel_path, :pkid)
        ON CONFLICT (pkid) DO NOTHING
    """)
    with engine.begin() as conn:
        conn.execute(q, {
            "contract_code": code,
            "inventory_year": AÑO,
            "inventory_date": "pending",
            "rel_path": rel_path,
            "pkid": pkid
        })
    print(f"✅ Insertado {code} ({rel_path}) para {AÑO}")


💻 Conectado a la base de datos helloworldtree
Total en batch: 107
Ya existen en DB: 68
Faltan por insertar: 72
       contract_code                                           rel_path
0             MX0021  Mexico/2023_ForestInventory/3-WT Cruises/Alvar...
2             MX0027  Mexico/2023_ForestInventory/3-WT Cruises/Carlo...
3             MX0028  Mexico/2023_ForestInventory/3-WT Cruises/Ciril...
5             MX0022  Mexico/2023_ForestInventory/3-WT Cruises/Grego...
6             MX0030  Mexico/2023_ForestInventory/3-WT Cruises/Herad...
..               ...                                                ...
102           CR0100  Costa Rica/2023_ForestInventory/3-WT Cruises/S...
103   Victor Godinez  Costa Rica/2023_ForestInventory/3-WT Cruises/V...
104   Wilbert Alfaro  Costa Rica/2023_ForestInventory/3-WT Cruises/W...
105   William Romero  Costa Rica/2023_ForestInventory/3-WT Cruises/W...
106  Wilman Alvarado  Costa Rica/2023_ForestInventory/3-WT Cruises/W...

[72 rows x 2 columns]
✅ 